# Paths

In [1]:
path = '/home/tih_isi_7/G-NeuroDAVIS/'
res_plots = path + 'Results/Plots/'
path_emb = path + 'Data/Embeddings/'
path_tab = path + 'Results/Tables/'

# Importing libraries

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append(path)

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import keras
import ipynbname
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from utils import Datasets as ds
from keras.models import load_model
from utils import DownstreamAnalysis as DA

dataname = ipynbname.name().rsplit("_")[0]
txt = %pwd
methodname = txt.rsplit("/")[-1]

# Load labels and embeddings

In [3]:
y = ds.LoadData(dataname, DType='None', LType=True)
x_embedding = np.array(pd.read_csv(path_emb+dataname+'/'+dataname+'_'+methodname+'.csv', index_col=0, header=0))

# Downstream analysis

### k-NN

In [4]:
neighbour = [5, 15, 25, 35, 45]
Score_knn = []
for n in neighbour:
    Score_knn.append(DA.Knn(x_embedding, y['x'], n))
Score_knn = pd.DataFrame(Score_knn, index=['neighbour '+ str(s) for s in neighbour],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_knn.to_csv(path_tab+dataname+'_'+methodname+'_knn.csv')
Score_knn

,Accuracy,Precision,Recall,F1-Score
neighbour 5,0.895833,0.901298,0.895833,0.894248
neighbour 15,0.833333,0.839674,0.833333,0.824411
neighbour 25,0.756944,0.751016,0.756944,0.739367
neighbour 35,0.729167,0.725035,0.729167,0.706321
neighbour 45,0.690972,0.696197,0.690972,0.660827


### Random Forest

In [5]:
n_tree = [20,40,60,80,100]
Score_rfc = []
for n in n_tree:
    Score_rfc.append(DA.RFC(x_embedding, y['x'], n))
Score_rfc = pd.DataFrame(Score_rfc, index=['n_tree '+ str(s) for s in n_tree],
                         columns = ['Accuracy', 'Precision','Recall','F1-Score'])
Score_rfc.to_csv(path_tab+dataname+'_'+methodname+'_rfc.csv')
Score_rfc

,Accuracy,Precision,Recall,F1-Score
n_tree 20,0.930556,0.934704,0.930556,0.930767
n_tree 40,0.920139,0.918987,0.920139,0.917730
n_tree 60,0.923611,0.922973,0.923611,0.921838
n_tree 80,0.930556,0.934856,0.930556,0.931007
n_tree 100,0.937500,0.940732,0.937500,0.937582


### k-Means

In [6]:
iteration = 10
Results_km = []

for i in tqdm(range(iteration), desc='Progress'):
    Results_km.append(DA.kmeans(x_embedding, y['x']))
    
Results_km = pd.DataFrame(Results_km,columns = ['ARI', 'FMI','NMI'])
Results_km

Progress: 100%|█████████████████████████████████| 10/10 [00:01<00:00,  8.78it/s]


,ARI,FMI,NMI
0,0.442631,0.658921,0.474356
1,0.431164,0.649775,0.463784
2,0.451478,0.670960,0.481589
3,0.453377,0.668104,0.482938
4,0.447808,0.669105,0.477895
5,0.436900,0.656304,0.467101
6,0.454149,0.668344,0.483658
7,0.417759,0.644410,0.449937
8,0.416088,0.646264,0.449695
9,0.423185,0.643324,0.454551


### Agglomerative 

In [ ]:
idx = ['ARI', 'FMI','NMI']
AglScore = pd.DataFrame(DA.Agglomerative(x_embedding, y['x']), index = idx)    #seurat_clusters, Cluster, celltype.l2
AglScore.T